# Term Project: Developing a Sentiment Analyzer
# The Multi-National Company chosen
# IMDb



# Install the necessary packages

In [1]:
!pip install pandas
!pip install numpy
!pip install nltk
!pip install sklearn


ERROR: Could not find a version that satisfies the requirement sklearn (from versions: none)
ERROR: No matching distribution found for sklearn


# Import the necessary libraries

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

# Load the dataset into a Pandas dataframe

In [3]:
df = pd.read_csv('IMDb.csv')

# Loaded data with labels

In [4]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Cleaning the data

# Remove duplicates

In [5]:
df.drop_duplicates(inplace=True)
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Remove rows with missing values

In [6]:
df.dropna(inplace=True)
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Remove URLS

In [7]:
import re
import pandas as pd
import re

df = pd.read_csv('IMDb.csv')

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

df['review'] = df['review'].apply(remove_urls)
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Remove Special Characters

In [8]:
def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9\s]' # keep only alphanumeric and whitespace characters
    text = re.sub(pattern, '', text)
    return text

df['review'] = df['review'].apply(remove_special_characters)
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production br br The filmin...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive
5,Probably my alltime favorite movie a story of ...,positive
6,I sure would like to see a resurrection of a u...,positive
7,This show was an amazing fresh innovative ide...,negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Convert text to lower case

In [9]:
df['review'] = df['review'].str.lower()
df.head(10)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production br br the filmin...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive
5,probably my alltime favorite movie a story of ...,positive
6,i sure would like to see a resurrection of a u...,positive
7,this show was an amazing fresh innovative ide...,negative
8,encouraged by the positive comments about this...,negative
9,if you like original gut wrenching laughter yo...,positive


# Tokenize the data

In [10]:
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# Tokenize text data
df['review_tokens'] = df['review'].apply(word_tokenize)
df.head(10)

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


,review,sentiment,review_tokens
0,one of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione..."
1,a wonderful little production br br the filmin...,positive,"[a, wonderful, little, production, br, br, the..."
2,i thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,..."
3,basically theres a family where a little boy j...,negative,"[basically, theres, a, family, where, a, littl..."
4,petter matteis love in the time of money is a ...,positive,"[petter, matteis, love, in, the, time, of, mon..."
5,probably my alltime favorite movie a story of ...,positive,"[probably, my, alltime, favorite, movie, a, st..."
6,i sure would like to see a resurrection of a u...,positive,"[i, sure, would, like, to, see, a, resurrectio..."
7,this show was an amazing fresh innovative ide...,negative,"[this, show, was, an, amazing, fresh, innovati..."
8,encouraged by the positive comments about this...,negative,"[encouraged, by, the, positive, comments, abou..."
9,if you like original gut wrenching laughter yo...,positive,"[if, you, like, original, gut, wrenching, laug..."


# Remove stopwords

In [11]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Load the dataset
df = pd.read_csv('IMDb.csv')

# Convert labels to numerical values (0 for negative, 1 for positive)
df['sentiment'] = df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

# Tokenize each review
df['tokenized_reviews'] = df['review'].apply(lambda x: x.lower().split())

# Remove stop words from each review
stop_words = set(stopwords.words('english'))
df['filtered_reviews'] = df['tokenized_reviews'].apply(lambda x: [word for word in x if word not in stop_words])

df.head(10)

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


,review,sentiment,tokenized_reviews,filtered_reviews
0,One of the other reviewers has mentioned that ...,1,"[one, of, the, other, reviewers, has, mentione...","[one, reviewers, mentioned, watching, 1, oz, e..."
1,A wonderful little production. <br /><br />The...,1,"[a, wonderful, little, production., <br, /><br...","[wonderful, little, production., <br, /><br, /..."
2,I thought this was a wonderful way to spend ti...,1,"[i, thought, this, was, a, wonderful, way, to,...","[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,0,"[basically, there's, a, family, where, a, litt...","[basically, there's, family, little, boy, (jak..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[petter, mattei's, ""love, in, the, time, of, m...","[petter, mattei's, ""love, time, money"", visual..."
5,"Probably my all-time favorite movie, a story o...",1,"[probably, my, all-time, favorite, movie,, a, ...","[probably, all-time, favorite, movie,, story, ..."
6,I sure would like to see a resurrection of a u...,1,"[i, sure, would, like, to, see, a, resurrectio...","[sure, would, like, see, resurrection, dated, ..."
7,"This show was an amazing, fresh & innovative i...",0,"[this, show, was, an, amazing,, fresh, &, inno...","[show, amazing,, fresh, &, innovative, idea, 7..."
8,Encouraged by the positive comments about this...,0,"[encouraged, by, the, positive, comments, abou...","[encouraged, positive, comments, film, looking..."
9,If you like original gut wrenching laughter yo...,1,"[if, you, like, original, gut, wrenching, laug...","[like, original, gut, wrenching, laughter, lik..."


# Training the sentiment analyzer using Naive Bayes, Support Vector Machine, and Logistic Regression

# Preprocess the data set

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the data set
data = pd.read_csv('IMDb.csv')

# Convert labels to numerical values

In [13]:
# Convert labels to numerical values (0 for negative, 1 for positive)
df['sentiment'] = df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)
df.head(10)

,review,sentiment,tokenized_reviews,filtered_reviews
0,One of the other reviewers has mentioned that ...,1,"[one, of, the, other, reviewers, has, mentione...","[one, reviewers, mentioned, watching, 1, oz, e..."
1,A wonderful little production. <br /><br />The...,1,"[a, wonderful, little, production., <br, /><br...","[wonderful, little, production., <br, /><br, /..."
2,I thought this was a wonderful way to spend ti...,1,"[i, thought, this, was, a, wonderful, way, to,...","[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,1,"[basically, there's, a, family, where, a, litt...","[basically, there's, family, little, boy, (jak..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[petter, mattei's, ""love, in, the, time, of, m...","[petter, mattei's, ""love, time, money"", visual..."
5,"Probably my all-time favorite movie, a story o...",1,"[probably, my, all-time, favorite, movie,, a, ...","[probably, all-time, favorite, movie,, story, ..."
6,I sure would like to see a resurrection of a u...,1,"[i, sure, would, like, to, see, a, resurrectio...","[sure, would, like, see, resurrection, dated, ..."
7,"This show was an amazing, fresh & innovative i...",1,"[this, show, was, an, amazing,, fresh, &, inno...","[show, amazing,, fresh, &, innovative, idea, 7..."
8,Encouraged by the positive comments about this...,1,"[encouraged, by, the, positive, comments, abou...","[encouraged, positive, comments, film, looking..."
9,If you like original gut wrenching laughter yo...,1,"[if, you, like, original, gut, wrenching, laug...","[like, original, gut, wrenching, laughter, lik..."


# Split the data into training and testing sets

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)
df.head(10)

,review,sentiment,tokenized_reviews,filtered_reviews
0,One of the other reviewers has mentioned that ...,1,"[one, of, the, other, reviewers, has, mentione...","[one, reviewers, mentioned, watching, 1, oz, e..."
1,A wonderful little production. <br /><br />The...,1,"[a, wonderful, little, production., <br, /><br...","[wonderful, little, production., <br, /><br, /..."
2,I thought this was a wonderful way to spend ti...,1,"[i, thought, this, was, a, wonderful, way, to,...","[thought, wonderful, way, spend, time, hot, su..."
3,Basically there's a family where a little boy ...,1,"[basically, there's, a, family, where, a, litt...","[basically, there's, family, little, boy, (jak..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,"[petter, mattei's, ""love, in, the, time, of, m...","[petter, mattei's, ""love, time, money"", visual..."
5,"Probably my all-time favorite movie, a story o...",1,"[probably, my, all-time, favorite, movie,, a, ...","[probably, all-time, favorite, movie,, story, ..."
6,I sure would like to see a resurrection of a u...,1,"[i, sure, would, like, to, see, a, resurrectio...","[sure, would, like, see, resurrection, dated, ..."
7,"This show was an amazing, fresh & innovative i...",1,"[this, show, was, an, amazing,, fresh, &, inno...","[show, amazing,, fresh, &, innovative, idea, 7..."
8,Encouraged by the positive comments about this...,1,"[encouraged, by, the, positive, comments, abou...","[encouraged, positive, comments, film, looking..."
9,If you like original gut wrenching laughter yo...,1,"[if, you, like, original, gut, wrenching, laug...","[like, original, gut, wrenching, laughter, lik..."


# Vectorize the text data

In [18]:
vectorizer = CountVectorizer(stop_words='english')
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


# 1. Train Naive Bayes model

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
df = pd.read_csv('IMDb.csv')

# Convert labels to numerical values (0 for negative, 1 for positive)
df['sentiment'] = df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

# Split the dataset into training and testing sets
train_df = df.sample(frac=0.8, random_state=1)
test_df = df.drop(train_df.index)

# Vectorize the reviews
vectorizer = CountVectorizer()
train_vectors = vectorizer.fit_transform(train_df['review'])
test_vectors = vectorizer.transform(test_df['review'])

# Train the Naive Bayes model
nb_classifier = MultinomialNB()
nb_classifier.fit(train_vectors, train_df['sentiment'])

# Predict the sentiments of the test set
test_preds = nb_classifier.predict(test_vectors)

# Calculate evaluation metrics
accuracy = accuracy_score(test_df['sentiment'], test_preds)
precision = precision_score(test_df['sentiment'], test_preds)
recall = recall_score(test_df['sentiment'], test_preds)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.8442
Precision: 0.8660276890308839
Recall: 0.814014014014014


#  2. Training the Support Vector Machine model 

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
df = pd.read_csv('IMDb.csv')

# Convert labels to numerical values (0 for negative, 1 for positive)
df['sentiment'] = df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Vectorize the text data using the bag-of-words approach
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a Logistic Regression model on the training data
lr = LogisticRegression(random_state=42)
lr.fit(X_train_vec, y_train)

# Predict the labels of the test data
y_pred = lr.predict(X_test_vec)

# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('Accuracy: {:.2f}'.format(accuracy))
print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))

C:\Users\User\anaconda3\New folder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.89
Precision: 0.89
Recall: 0.90


# 3. Train Logistic Regression model

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
df = pd.read_csv('IMDb.csv')

# Convert labels to numerical values (0 for negative, 1 for positive)
df['sentiment'] = df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Vectorize the text data using the bag-of-words approach
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a Logistic Regression model on the training data
lr = LogisticRegression(random_state=42)
lr.fit(X_train_vec, y_train)

# Predict the labels of the test data
y_pred = lr.predict(X_test_vec)

# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('Accuracy: {:.2f}'.format(accuracy))
print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))

C:\Users\User\anaconda3\New folder\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.89
Precision: 0.89
Recall: 0.90


# Loading the trained Support Vector Machine model

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Load the training data from a CSV file
train_data = pd.read_csv('IMDb.csv')

# Define the vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data
vectorizer.fit(train_data['review'])

# Transform the training data
train_vectors = vectorizer.transform(train_data['review'])

# Define the LinearSVC model
svm_model = LinearSVC()

# Train the model on the training data
svm_model.fit(train_vectors, train_data['sentiment'])

# Process the new review
review = "Wow, what a fantastic movie—I really enjoyed it!"
new_review_vector = vectorizer.transform([review])

# Predict the sentiment of the new review
new_review_sentiment = svm_model.predict(new_review_vector)

# Print the sentiment of the new review
print("The sentiment of this reviw is: " + new_review_sentiment[0])

The sentiment of this reviw is: positive


# Deploying the model

In [35]:
import gradio as gr

def predict_sentiment(review):
    # Process the new review
    new_review_vector = vectorizer.transform([review])

    # Predict the sentiment of the new review
    new_review_sentiment = svm_model.predict(new_review_vector)

    return new_review_sentiment[0]

review_input = gr.inputs.Textbox(lines=3, label="Enter your review")
prediction = gr.outputs.Label(label="The Sentiment of this review is : ")

gradio_interface = gr.Interface(fn=predict_sentiment, inputs=review_input, outputs=prediction, title='Sentiment Analyzer by Aywa Stonny', 
             description='Enter a piece of text to get its sentiment')
gradio_interface.launch(share=True)

C:\Users\User\AI\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\User\AI\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\User\AI\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\User\AI\gradio\outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\User\AI\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app
